<a href="https://colab.research.google.com/github/Yuto-T-440/25s-MatDataSci-Tohoku/blob/main/GNN_modeling2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install torch torchvision torchaudio
!pip install torch-geometric
!pip install rdkit-pypi

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_mean_pool

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
import numpy as np
from torch_geometric.data import Data
from torch_geometric.nn import radius_graph
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool
from torch.optim import Adam
import torch
import torch.nn.functional as F
from torch.nn import Linear, Module
from sklearn.preprocessing import StandardScaler

In [ ]:
def build_graph(Z, R, y):
    x = torch.tensor(Z, dtype=torch.float).reshape(-1, 1)
    pos = torch.tensor(R, dtype=torch.float)

    y = torch.tensor(y, dtype=torch.float).reshape(1, -1)  # ✅ fix: ensures [1, 6] shape

    edge_index = radius_graph(pos, r=5.0, loop=False)
    return Data(x=x, pos=pos, edge_index=edge_index, y=y)

In [ ]:
# Load the .npz file
data = np.load('/content/drive/MyDrive/InformationTechnologyFundamental/DFT_all.npz',allow_pickle=True)

In [ ]:
epoch_max = 50 #Number of epochs we train the network

In [ ]:
# Initialize lists to store metrics for plotting
train_losses, val_losses, val_mae_totals, val_mae_per_target = [], [], [], []

Zs = data['atoms']             # atomic numbers (per molecule)
Rs = data['coordinates']       # 3D positions (per molecule)

In [ ]:
# Extract dipole components
dipoles = data['dipole']
dipole_x = np.array([d[0] for d in dipoles])
dipole_y = np.array([d[1] for d in dipoles])
dipole_z = np.array([d[2] for d in dipoles])

# Define targets
targets = ['U0', 'gap', 'H']
Y_all = np.column_stack([
    data['U0'], data['gap'], data['H'],
    dipole_x, dipole_y, dipole_z
])
scaler = StandardScaler()
Y_scaled = scaler.fit_transform(Y_all)
Ys = list(Y_scaled)

In [ ]:
# 🔽 NEW: Split the data into train/val/test
from sklearn.model_selection import train_test_split

Z_train, Z_temp, R_train, R_temp, Y_train, Y_temp = train_test_split(Zs, Rs, Ys, test_size=0.2, random_state=42)
Z_val, Z_test, R_val, R_test, Y_val, Y_test = train_test_split(Z_temp, R_temp, Y_temp, test_size=0.5, random_state=42)

In [ ]:
# PyTorch Geometric & 依存ライブラリのインストール（torch-cluster含む）
!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric \
  -f https://data.pyg.org/whl/torch-2.6.0+cu124.html

Looking in links: https://data.pyg.org/whl/torch-2.6.0+cu124.html


In [ ]:
from torch_cluster import radius_graph, knn_graph

In [ ]:
# 🔽 Build separate datasets
train_dataset = [build_graph(z, r, y) for z, r, y in zip(Z_train, R_train, Y_train)]
val_dataset   = [build_graph(z, r, y) for z, r, y in zip(Z_val, R_val, Y_val)]
test_dataset  = [build_graph(z, r, y) for z, r, y in zip(Z_test, R_test, Y_test)]

In [ ]:
# 🔽 Use separate data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader  = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
dataset = [build_graph(z, r, y) for z, r, y in zip(Zs, Rs, Ys)]
loader = DataLoader(dataset, batch_size=32, shuffle=True)

In [ ]:
class GNNModel(Module):
    def __init__(self, hidden_dim=64, output_dim=6):
        super().__init__()
        self.conv1 = GCNConv(1, hidden_dim)  # input: atomic number
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.lin1 = Linear(hidden_dim, hidden_dim)
        self.lin2 = Linear(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = global_mean_pool(x, batch)       # Global graph representation
        x = F.relu(self.lin1(x))
        out = self.lin2(x)                   # Multi-target output
        return out

In [ ]:
from sklearn.metrics import mean_absolute_error

In [ ]:
import os

# フォルダがなければ作成
os.makedirs("saves", exist_ok=True)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GNNModel().to(device)
optimizer = Adam(model.parameters(), lr=0.001)
loss_fn = torch.nn.MSELoss()

for epoch in range(1, epoch_max+1):
    model.train()
    total_loss = 0
    for batch in train_loader:
        batch = batch.to(device)
        pred = model(batch)
        target = batch.y
        loss = loss_fn(pred, target)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        total_loss += loss.item() * batch.num_graphs

    avg_train_loss = total_loss / len(train_loader.dataset)
    train_losses.append(avg_train_loss)

    # ===== Validation starts here =====
    model.eval()
    val_loss = 0
    val_preds, val_targets = [], []

    with torch.no_grad():
        for batch in val_loader:
            batch = batch.to(device)
            pred = model(batch)
            target = batch.y
            val_loss += loss_fn(pred, target).item() * batch.num_graphs

            val_preds.append(pred.cpu().numpy())
            val_targets.append(target.cpu().numpy())

    avg_val_loss = val_loss / len(val_loader.dataset)
    val_losses.append(avg_val_loss)

    # ==== MAE (real units)
    val_preds_real = scaler.inverse_transform(np.vstack(val_preds))
    val_targets_real = scaler.inverse_transform(np.vstack(val_targets))

    val_mae_real = mean_absolute_error(val_targets_real, val_preds_real)
    val_mae_totals.append(val_mae_real)

    # ==== Per-target MAEs
    per_target_mae = []
    for i in range(val_targets_real.shape[1]):
        mae_i = mean_absolute_error(val_targets_real[:, i], val_preds_real[:, i])
        per_target_mae.append(mae_i)

    val_mae_per_target.append(per_target_mae)

    # Format per-target MAEs for display
    target_names = ['U0', 'gap', 'H', 'dip_x', 'dip_y', 'dip_z']
    mae_str = " | ".join([f"{name}: {mae:.4f}" for name, mae in zip(target_names, per_target_mae)])

    print(f"Epoch {epoch:02d} | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f} | Val MAE (total): {val_mae_real:.4f}")
    print(f"  MAE per target: {mae_str}")

torch.save(model,"saves/model.nn")
np.save("saves/val_mae_totals.npy",val_mae_totals,allow_pickle = True)
np.save("saves/val_mae_per_target.npy",val_mae_per_target,allow_pickle = True)
np.save("saves/val_losses.npy",val_losses,allow_pickle = True)
np.save("saves/train_losses.npy",train_losses,allow_pickle = True)

Epoch 01 | Train Loss: 0.7756 | Val Loss: 0.7740 | Val MAE (total): 126.8535
  MAE per target: U0: 379.0259 | gap: 0.0394 | H: 378.7517 | dip_x: 1.4134 | dip_y: 1.0780 | dip_z: 0.8089
Epoch 02 | Train Loss: 0.7450 | Val Loss: 0.7358 | Val MAE (total): 107.8821
  MAE per target: U0: 322.0344 | gap: 0.0386 | H: 321.9182 | dip_x: 1.4109 | dip_y: 1.0780 | dip_z: 0.8069
Epoch 03 | Train Loss: 0.7301 | Val Loss: 0.7334 | Val MAE (total): 108.6703
  MAE per target: U0: 324.5230 | gap: 0.0384 | H: 324.1676 | dip_x: 1.4099 | dip_y: 1.0780 | dip_z: 0.8077
Epoch 04 | Train Loss: 0.7234 | Val Loss: 0.7258 | Val MAE (total): 101.7161
  MAE per target: U0: 303.4839 | gap: 0.0382 | H: 303.4827 | dip_x: 1.4111 | dip_y: 1.0785 | dip_z: 0.8075
Epoch 05 | Train Loss: 0.7206 | Val Loss: 0.7233 | Val MAE (total): 100.9357
  MAE per target: U0: 301.1976 | gap: 0.0383 | H: 301.0880 | dip_x: 1.4091 | dip_y: 1.0786 | dip_z: 0.8068
Epoch 06 | Train Loss: 0.7192 | Val Loss: 0.7199 | Val MAE (total): 100.2404
  M